# Markov Chains
**Using Probability to Generate Text**

By Jade Tabony


### What are Markov Chains?
Markov chains describe a stochastic process that is used to predict a series of events, or a chain, if you will.  The occurence of each event is dependent only on the current state of the system.  Because of this, Markov chains are said to be "Memoryless".  

#### So what is happening?

![chain](img/two-cities.jpg)
http://oatzy.blogspot.com/2011/04/markovs-next-tweet.html

When applied to building sentences, this means that as the system is constructing the sentence, word by word, the next word is decided only on the word preceding it, having nothing to do with the rest of the sentence.  This can lead to some really interesting results.

Examples:

- https://twitter.com/captain_markov
- https://twitter.com/markovchainme
- https://twitter.com/scifri_ebooks
- https://twitter.com/Pliny_theElder

![xkcd](img/xkcd.png)
https://xkcd.com/1068/

So lets build one!!

--------------------------------------------------------------------------

### Lewis Carroll goes for a Hike.

![chesirecat](img/chesire_cat.png)

For my example, I'm going to train my chain on Lewis Carroll books and hike descriptions from the Washington Trails Association.

First off, import your necessary libraries.  If you do not have anaconda installed, you may have to pip install some of the libraries.  Pandas was necessary for my example, since my hike data was stored in a csv file, but you may not need it.

In [29]:
import random
import string
import pandas as pd

You're going to need to text to "train" your model on.  My text was downloaded from Project Gutenberg and scraped from the WTA website

In [30]:
alice_text = open('data/AliceInWonderland.txt', 'r').read()

In [31]:
hikes = pd.read_csv('data/washington_hikes_clean_noout.csv')
hikes = hikes[['hike_name', 'hike_desc']].dropna()
hikes['desc_len'] = [len(text) for text in hikes['hike_desc']]

Now to build the code for our Markov Chain bot.  I've decided to build it with the potential for both bigram and single word seeds. Some of this code was borrowed from [this blog post](https://sookocheff.com/post/nlp/ngram-modeling-with-markov-chains/), which I customized to include the potential for training on both bigrams and trigrams, and to include the tweeting functionality

In [32]:
class MarkovChain(object):

    def __init__(self):
        self.memory = {}
        self.bigram_memory = {}

    def _learn_key(self, key, value):
        if key not in self.memory:
            self.memory[key] = []

        self.memory[key].append(value)
    
    def _learn_bigramkey(self, key, value):
        if key not in self.memory:
            self.bigram_memory[key] = []

        self.bigram_memory[key].append(value)

    def learn(self, text, bigram_keys=True):
        #tokens = [item for item in map(string.strip, re.split("(\W+)", text)) if len(item) > 0]
        tokens = text.split()
        if bigram_keys: 
            trigrams = [(tokens[i], tokens[i + 1], tokens[i + 2]) for i in range(0, len(tokens) - 2)]
            for trigram in trigrams:
                self._learn_bigramkey((trigram[0], trigram[1]), trigram[2])
        else: 
            bigrams = [(tokens[i], tokens[i + 1]) for i in range(0, len(tokens) - 1)]
            for bigram in bigrams:
                self._learn_key(bigram[0], bigram[1])

    def _next(self, current_state):
        next_possible = self.memory.get(current_state)

        if not next_possible:
            next_possible = self.memory.keys()

        return random.sample(next_possible, 1)[0]
    
    def _next_bigram(self, current_state):
        next_possible = self.bigram_memory.get(current_state)

        if not next_possible:
            next_possible = self.bigram_memory.keys()

        return random.sample(next_possible, 1)[0]
    
    def tweet(self, min_length=70, max_length=140, use_bigram=True):
        tweet_length = random.randint(min_length, max_length)
        tweet = []
        if use_bigram:
            seed = random.choice(self.bigram_memory.keys())
            tweet.append(seed[0])
            tweet.append(seed[1])
            tweet_length -= len(seed) 
            while tweet_length>0:
                next_word = self._next_bigram((tweet[-2], tweet[-1]))
                tweet.append(next_word)
                tweet_length-=len(next_word)
        else: 
            seed = random.choice(self.memory.keys())
            tweet.append(seed)
            tweet_length -= len(seed) 
            while tweet_length>0:
                next_word = self._next(tweet[-1])
                tweet.append(next_word)
                tweet_length-=len(next_word)
        tweet[0] = tweet[0].capitalize()
        tweet[-1] = tweet[-1].strip(string.punctuation) + random.choice(['.', '!', '?'])
        return " ".join(tweet)
            
        
        

Once you have your Markov Chain bot class built, you can initialize, train and tweet with it!

In [33]:
lewis = MarkovChain()

In [34]:
lewis.learn(alice_text)

In [36]:
lewis.tweet()

'Over. Alice was too much frightened to say a word, but slowly followed her back to the White Rabbit read out the verses!'

Now lets add some hike descriptions!  These descriptions were scraped from the WTA website using requests and BeautifulSoup for another project. 

In [37]:
for hike in hikes['hike_desc']:
    lewis.learn(hike)

In [38]:
lewis.tweet()

'Gain 400 feet below, while far beyond the rich foliage and wildlife habitat of chukar--a beautiful upland game bird that prefers steep slopes and arid landscapes.'

In [39]:
lewis.tweet()

'Quickly encounter the Hidden Forest Trail uphill, at first climbing steeply then leveling out. After 0.2 miles across a grassy rock slab?'

Hmmm... Now most of my tweets seem to be dominated by WTA text. Let's check how long each body of text is.

In [40]:
len(alice_text)

53024

In [41]:
sum(hikes.desc_len)

2596937

Turns out, most of my memory consists of WTA text.  So I'll add more Lewis.

In [42]:
looking_glass = open('data/ThroughTheLookingGlass.txt', 'r').read()
len(looking_glass)

162063

In [43]:
lewis.learn(looking_glass)

In [47]:
lewis.tweet()

'And second, a rocky outcrop.In summer, as you can, And sprinkle the table till she had found the Red King, Kitty?'

**BETTER!!!**

![grumpy-cat](img/grumpy_cat.jpg)

### Other things you might consider:
- Training your Markov bot to start in a logical spot. (But logic is boring)
- Similarly, find a logical ending. (see previous point)
- If you're training using multiple bodies of text of significantly differing length, consider under or oversampling.
- Ensuring that opened quotes or parentheses close. 

## Markovify 
#### Just in case you don't want to write this from scratch


https://github.com/jsvine/markovify

In [1]:
import markovify

with open("data/AliceInWonderland.txt") as f:
    text = f.read()
    
    
text_model = markovify.Text(text)

for i in range(5):
    print(text_model.make_sentence())

To Alice's great surprise, the Duchess's arm that was sitting on the back.
This did not venture to go down the chimney as she was nine feet high, and she felt certain it must be really offended.
He came in sight of the house and found that she was shrinking rapidly; so she set to work at once to eat the comfits; this caused some noise and confusion, as the Rabbit and had to kneel down on the bank, and of having nothing to do.
The judge, by the White Rabbit, with a little shriek and a large dish of tarts upon it.
She looked up, but it was the King and he wore his crown over his great wig.


## MarkovBot!
Just in case you want to do even less

http://www.pygaze.org/2016/03/how-to-code-twitter-bot/

### RESOURCES

- [Project Gutenberg](https://www.gutenberg.org/)
- [The Washington Trails Association](https://wta.org)
- https://sookocheff.com/post/nlp/ngram-modeling-with-markov-chains/
- http://www.onthelambda.com/2014/02/20/how-to-fake-a-sophisticated-knowledge-of-wine-with-markov-chains/